# Reddit Crypto Sentiment Dashboard

In [ ]:
#This Python script fetches cryptocurrency discussions from Reddit's r/cryptocurrency subreddit, performs sentiment analysis, 
#and ranks the top 15 most positively mentioned coins. The results are displayed in the console and stored in Google Sheets for further analysis.

#Key Features
#--------#
# Fetches up to 500 posts & comments from r/cryptocurrency (last 30 days), analyzes sentiment using VADER, and filters only positive discussions (compound score >= 0.2).
# Extracts crypto symbols ($BTC, $ETH, $SOL) using regex, groups data, and ranks the top 15 coins based on positive mentions & average sentiment.
# Saves results in Google Sheets by clearing old data and updating with new insights, while displaying a formatted table of top positive sentiment coins in the console. 

#Workflo


In [7]:
import praw
import re
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from tabulate import tabulate
from datetime import datetime, timedelta, timezone
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 🔹 Reddit API Connection
reddit = praw.Reddit(
    client_id='u8KF23yuLUQhHugEnsLMCQ',
    client_secret='RjLKMo6By4zcQKKDscGTgBiRs44Zwg',
    user_agent='MyFirstApp/1.0 by u/Appropriate_Light976's
)

# 🔹 Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# 🔹 Google Sheets Authentication
def connect_google_sheets(sheet_name):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open(sheet_name).sheet1  # Select the first sheet
    return sheet

# 🔹 Fetch Reddit Data
def fetch_reddit_data(subreddit_name):
    """Fetches posts and comments from a subreddit for the last 30 days."""
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    thirty_days_ago = datetime.now(timezone.utc) - timedelta(days=30)

    for submission in subreddit.hot(limit=500):  # Fetch up to 500 posts
        post_datetime = datetime.fromtimestamp(submission.created_utc, timezone.utc)
        if post_datetime >= thirty_days_ago:
            post = {
                'Title': submission.title,
                'Post Body': submission.selftext,
                'Comments': " ".join([comment.body for comment in submission.comments.list() if hasattr(comment, 'body')]),
                'Date': post_datetime.strftime('%Y-%m-%d'),  # Only Date
                'Time': post_datetime.strftime('%H:%M:%S')   # Only Time
            }
            data.append(post)
    return pd.DataFrame(data)

# 🔹 Sentiment Analysis
def sentiment_score(text):
    """Calculates sentiment polarity using VADER."""
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# 🔹 Extract Coin Mentions
def find_coin_mentions(text):
    """Identifies cryptocurrency mentions."""
    pattern = r"\$[A-Z]+"  # Matches symbols like $BTC, $ETH, $SOL
    return re.findall(pattern, text)

# 🔹 Analyze Positive Sentiment
def analyze_positive_sentiment(data):
    """Analyzes data for coin mentions with positive sentiment."""
    data['Combined Text'] = data['Title'] + ' ' + data['Post Body'] + ' ' + data['Comments']
    data['Sentiment'] = data['Combined Text'].apply(sentiment_score)
    data['Coin Mentions'] = data['Combined Text'].apply(find_coin_mentions)

    exploded_data = data.explode('Coin Mentions').dropna(subset=['Coin Mentions'])
    positive_data = exploded_data[exploded_data['Sentiment'] >= 0.2]

    positive_summary = positive_data.groupby(['Coin Mentions', 'Date', 'Time']).agg(
        Count=('Coin Mentions', 'size'),
        Average_Positive_Sentiment=('Sentiment', 'mean')
    ).reset_index()

    positive_summary = positive_summary.sort_values(by='Count', ascending=False)

    # 🔹 Get the Top 15 Most Positively Mentioned Coins
    top_15_positive = positive_summary.head(15)

    return top_15_positive

# 🔹 Upload Results to Google Sheets
def upload_to_google_sheets(dataframe, sheet_name):
    sheet = connect_google_sheets(sheet_name)
    sheet.clear()
    
    sheet.append_row(["Coin", "Date", "Time", "Positive Mentions", "Avg Positive Sentiment"])
    
    for row in dataframe.itertuples(index=False):
        sheet.append_row(list(row))

# 🔹 Function to Update Dashboard
def update_dashboard(sheet_name, subreddit_name):
    print("🔄 Fetching Reddit data from the last 30 days...")
    data = fetch_reddit_data(subreddit_name)

    if data.empty:
        print("❌ No data found for the last 30 days.")
        return

    print("🔎 Analyzing data for positive sentiment...")
    positive_summary = analyze_positive_sentiment(data)

    if positive_summary.empty:
        print("⚠️ No coins with positive sentiment found.")
    else:
        print("\n📊 **Top 15 Coins with Positive Sentiment (Last 30 Days):**")
        print(tabulate(
            positive_summary,
            headers=['Coin', 'Date', 'Time', 'Positive Mentions', 'Avg Positive Sentiment'],
            tablefmt='fancy_grid'
        ))
        
        print("\n📤 Updating Google Sheets Dashboard...")
        upload_to_google_sheets(positive_summary, sheet_name)
        print("✅ Google Sheets updated successfully.")

if __name__ == "__main__":
    sheet_name = "Reddit Crypto Sentiment Dashboard"
    subreddit_name = "cryptocurrency"
    update_dashboard(sheet_name, subreddit_name)


🔄 Fetching Reddit data from the last 30 days...
🔎 Analyzing data for positive sentiment...

📊 **Top 15 Coins with Positive Sentiment (Last 30 Days):**
╒════╤══════════╤════════════╤══════════╤═════════════════════╤══════════════════════════╕
│    │ Coin     │ Date       │ Time     │   Positive Mentions │   Avg Positive Sentiment │
╞════╪══════════╪════════════╪══════════╪═════════════════════╪══════════════════════════╡
│ 35 │ $TRUMP   │ 2025-02-04 │ 20:26:31 │                   3 │                   0.9996 │
├────┼──────────┼────────────┼──────────┼─────────────────────┼──────────────────────────┤
│  2 │ $ETH     │ 2025-02-03 │ 00:01:17 │                   3 │                   0.9888 │
├────┼──────────┼────────────┼──────────┼─────────────────────┼──────────────────────────┤
│ 38 │ $TRUMP   │ 2025-02-05 │ 21:07:40 │                   2 │                   0.9081 │
├────┼──────────┼────────────┼──────────┼─────────────────────┼──────────────────────────┤
│ 27 │ $T       │ 2025-02-04 │